Download the fabfile from GitHub and check the head. Now I use an Ubuntu box with IP address and root password emailed by DigitalOcean.

In [2]:
!wget https://raw.githubusercontent.com/dapangmao/Blog/master/Deploy%20edx%20spark%20environment%20to%20DigitalOcean/fabfile.py
!head fabfile.py

--2015-07-28 21:16:52--  https://raw.githubusercontent.com/dapangmao/Blog/master/Deploy%20edx%20spark%20environment%20to%20DigitalOcean/fabfile.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 23.235.46.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|23.235.46.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4085 (4.0K) [text/plain]
Saving to: 'fabfile.py'

100%[======================================>] 4,085       --.-K/s   in 0s      

2015-07-28 21:16:53 (363 MB/s) - 'fabfile.py' saved [4085/4085]

from fabric.api import cd, env, puts, sudo, run
from fabric.contrib.files import exists

env.hosts = ['104.236.12.85']
env.user = 'root'
env.password = '12345'

PROJECT = 'ipython-spark'
CURRENT_VERSION = 'spark-1.4.1-bin-hadoop2.6'
CURRENT_URL = 'http://d3kbcqa49mib13.cloudfront.net/' + CURRENT_VERSION + '.tgz'


There are a few functions or commands in this fabfile. They could be sequentially run or called by `deploy_edx_spark` once for all.

In [3]:
!fab -l

Available commands:

    adjust_supervisor
    deploy_edx_spark
    download_edx_data
    download_spark
    get_pwd_hash
    install_basics
    make_ssl
    run_ipython


####Step 1. Install all required packages including IPython. 

In [10]:
!fab install_basics

[104.236.12.85] Executing task 'install_basics'
[104.236.12.85] sudo: apt-get -q update
[104.236.12.85] out: Ign http://security.ubuntu.com vivid-security InRelease
[104.236.12.85] out: Hit http://security.ubuntu.com vivid-security Release.gpg
[104.236.12.85] out: Hit http://security.ubuntu.com vivid-security Release
[104.236.12.85] out: Hit http://security.ubuntu.com vivid-security/main Sources
[104.236.12.85] out: Hit http://security.ubuntu.com vivid-security/universe Sources
[104.236.12.85] out: Hit http://security.ubuntu.com vivid-security/main amd64 Packages
[104.236.12.85] out: Hit http://security.ubuntu.com vivid-security/universe amd64 Packages
[104.236.12.85] out: Hit http://security.ubuntu.com vivid-security/main i386 Packages
[104.236.12.85] out: Hit http://security.ubuntu.com vivid-security/universe i386 Packages
[104.236.12.85] out: Hit http://security.ubuntu.com vivid-security/main Translation-en
[104.236.12.85] out: Hit http://security.ubuntu.com vivid-security/universe 

####Step 2. Issue self-signed SSL.

In [11]:
!fab make_ssl

[104.236.12.85] Executing task 'make_ssl'
[104.236.12.85] sudo: openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mycert.pem -out mycert.pem -subj "/CN=edx/O=spark/C=US"
[104.236.12.85] out: Generating a 1024 bit RSA private key
[104.236.12.85] out: ....................++++++
[104.236.12.85] out: ................................................++++++
[104.236.12.85] out: writing new private key to 'mycert.pem'
[104.236.12.85] out: -----
[104.236.12.85] out: 


Done.
Disconnecting from 104.236.12.85... done.


####Step 3. Download the lastest Spark

In [12]:
!fab download_spark

[104.236.12.85] Executing task 'download_spark'

Done.
Disconnecting from 104.236.12.85... done.


####Step 4. Download the data and IPython notebook from the edx courses cs100 and cs190

In [13]:
!fab download_edx_data

[104.236.12.85] Executing task 'download_edx_data'
[104.236.12.85] run: wget https://github.com/spark-mooc/cs100-data/raw/master/cs100.zip
[104.236.12.85] out: --2015-07-28 20:53:59--  https://github.com/spark-mooc/cs100-data/raw/master/cs100.zip
[104.236.12.85] out: Resolving github.com (github.com)... 192.30.252.129
[104.236.12.85] out: Connecting to github.com (github.com)|192.30.252.129|:443... connected.
[104.236.12.85] out: HTTP request sent, awaiting response... 302 Found
[104.236.12.85] out: Location: https://raw.githubusercontent.com/spark-mooc/cs100-data/master/cs100.zip [following]
[104.236.12.85] out: --2015-07-28 20:53:59--  https://raw.githubusercontent.com/spark-mooc/cs100-data/master/cs100.zip
[104.236.12.85] out: Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.27.76.133
[104.236.12.85] out: Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.27.76.133|:443... connected.
[104.236.12.85] out: HTTP request sent, awaiting response

####Step 5. Setup up the access password. Here the password is a simple 12345.

In [14]:
!fab get_pwd_hash:"12345"

[104.236.12.85] Executing task 'get_pwd_hash'
[104.236.12.85] run: echo 'from IPython.lib import passwd; print passwd(str(12345))' > pwd.py
[104.236.12.85] run: python pwd.py
[104.236.12.85] out: sha1:7d534c14e42e:e147d921d310b397b0a99eb80f4bd7870221f4d2
[104.236.12.85] out: 

[104.236.12.85] run: ipython profile create nbserver
[104.236.12.85] run: echo '
# Kernel config
c = get_config()
# Notebook config
c.NotebookApp.certfile = u"/root/.ipython/mycert.pem"
c.NotebookApp.keyfile = u"/root/.ipython/mycert.pem"
c.NotebookApp.ip = "*"
c.NotebookApp.open_browser = False
c.NotebookApp.password = u"sha1:7d534c14e42e:e147d921d310b397b0a99eb80f4bd7870221f4d2"
# Set up the port
c.NotebookApp.port = 9999
c.NotebookApp.notebook_dir = u"/root"
' >> ipython_notebook_config.py
[104.236.12.85] run: echo 'c.IPKernelApp.matplotlib = "inline"' >> ipython_config.py

Done.
Disconnecting from 104.236.12.85... done.


####Step 6. Set IPython notebook as a permenant process.

In [15]:
!fab adjust_supervisor

[104.236.12.85] Executing task 'adjust_supervisor'
[104.236.12.85] sudo: supervisorctl reread
[104.236.12.85] out: No config updates to processes
[104.236.12.85] out: 

[104.236.12.85] sudo: supervisorctl update

Done.
Disconnecting from 104.236.12.85... done.


####Step 7. Fire up the IPython server.

In [17]:
!fab run_ipython

[104.236.12.85] Executing task 'run_ipython'
[104.236.12.85] sudo: supervisorctl start ipython-spark
[104.236.12.85] out: ipython-spark: started
[104.236.12.85] out: 

[104.236.12.85] Now go to https://104.236.12.85:9999 to view the app

Done.
Disconnecting from 104.236.12.85... done.
